In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *


import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
!python /apps/open_spiel/open_spiel/python/examples/ubc_mccfr_cpp_example.py --solver cfr --filename rw_game_2_settle_spiteful.json --iterations 1000 --output ./neil

Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/rw_game_2_settle_spiteful.json
Done config parsing
I0517 22:54:45.857346 139852049412224 ubc_mccfr_cpp_example.py:169] Using C++ implementations
I0517 22:54:45.857650 139852049412224 ubc_mccfr_cpp_example.py:171] Using CFR solver
I0517 22:54:45.868486 139852049412224 ubc_mccfr_cpp_example.py:228] Starting iteration 0
I0517 22:54:45.984766 139852049412224 ubc_mccfr_cpp_example.py:265] Updated best policy on iteration 0
I0517 22:54:45.985294 139852049412224 ubc_mccfr_cpp_example.py:268] Iteration 0
I0517 22:54:45.985397 139852049412224 ubc_mccfr_cpp_example.py:272] walltime=0.031359
I0517 22:54:45.985480 139852049412224 ubc_mccfr_cpp_example.py:272] max_on_path_regret=0.574654
I0517 22:54:45.985557 139852049412224 ubc_mccfr_cpp_example.py:272] nash_conv=1.149248
I0517 22:54:46.174520 139852049412

In [5]:
df = pd.read_csv('./neil/reduced_strategy.csv')

In [7]:
df.query('player == 0 and round == 1')

,info_state,player,Allocation 0 A,Allocation 1 A,Bid (0 A),Bid (1 A),Bid (2 A),Price A,Utility 0,Utility 1,budget,my_bids,prob,round,terminal,total_demand,value
0,"Player 0\nMarginalValues:101, 51 Budget: 200\n",0,NaN,NaN,0.0,1.0,0.0,10.0,NaN,NaN,NaN,NaN,1.0,1,False,NaN,NaN


In [ ]:
q = df.query('terminal and player == 0')
q['prob'] @ q['Utility 0']

In [ ]:
# df.query('not terminal').query('player == 0').query('round == 18').sort_values('prob', ascending=False)
with pd.options('make columns bigger'):
    df.query('terminal').query('player == 0').sort_values('prob', ascending=False)

In [ ]:
df = pd.read_csv('./neil/reduced_strategy.csv')
df.query('terminal').query('player == 0').sort_values('prob', ascending=False)
q = df.query('terminal and player == 0')
q['prob'] @ q['Utility 0']

In [ ]:
game_name = 'very_large_game_2.json'
game = smart_load_clock_auction(game_name)
game_config = load_game_config(game_name)




In [ ]:
print(k_max)

In [ ]:
n_chance = game.max_chance_outcomes()
n_actions = game.num_distinct_actions()
k_max = max(n_chance, n_actions) + 1 # +1 for parent

n_steps = 0
N_STEPS_MAX = 10_000
state = game.new_initial_state()
action_hist = []
state_hist = []

randomness = np.random.rand(N_STEPS_MAX * 10000)
rand_index = 0


In [ ]:
def flip_coin(p):
    rand_index += 1
    return randomness[rand_index] < p

failure_count = 0
while n_steps < N_STEPS_MAX:
    stay = True
    node_degree = 1 # Parent
    if state.is_chance_node():
        legal_actions = state.chance_outcomes()
    else:
        legal_actions = state.legal_actions()
    node_degree += len(legal_actions)

    # Should I stay or should I go?
    # Stay at node 𝑛 with probability 1−𝑘𝑛/𝑘max
    stay_prob = 1 - (node_degree / k_max)
    while stay:
        stay = flip_coin(stay_prob)
        if stay:
            n_steps += 1 
            continue
    else:
        if state.is_initial_state():
            transition_to_parent = False
        else:
             transition_to_parent = flip_coin(1 / node_degree)
        if transition_to_parent:
            state = state_hist.pop()
            action = action_hist.pop()
            n_steps += 1
        else:
            if state.is_chance_node():
                outcomes = state.chance_outcomes()
                action_list, prob_list = zip(*outcomes)
                action = np.random.choice(action_list, p=prob_list)
            else:
                action = random.choice(state.legal_actions(state.current_player()))
            # Only go through with this if it's not terminal
            clone = state.clone()
            clone.apply_action(action)
            if not clone.is_terminal(): # TODO: Could be smarter about resmampling here by taking away the same choices
                state_hist.append(state.clone())
                state = clone
                action_hist.append(action)
            else:
                failure_count += 1

# print(state)

In [ ]:
print(game.new_initial_state())

In [ ]:
state.clone()

In [ ]:
from auctions.webutils import *

env_and_model = db_checkpoint_loader(EquilibriumSolverRunCheckpoint.objects.last())

In [ ]:
env_and_model.agents[0]._rl_agent._get_epsilon(False)

In [ ]:
np.random.binomial(1, 0.9)

In [ ]:
import torch